In [8]:
import pandas as pd
import numpy as np
import heapq
import pprint

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import defaultdict


### 1. Use the contents of ratings.csv to create a dataset for your network. There should be a feature vector for each user, with each feature corresponding to a movie. Encode movies that the user has rated 3.0 or above as +1, and other movies as -1.

In [9]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2021-04-27 16:48:07--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2021-04-27 16:48:07 (8.00 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [10]:
!unzip -n ml-latest-small.zip

Archive:  ml-latest-small.zip


In [11]:
!ls

ml-latest-small  ml-latest-small.zip  sample_data


In [27]:
rating_df = pd.read_csv('ml-latest-small/ratings.csv')
rating_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [28]:
rating_df['encode'] = np.where(rating_df['rating'] >= 3.0, 1, -1)

In [29]:
rating_df.head()

,userId,movieId,rating,timestamp,encode
0,1,1,4.0,964982703,1
1,1,3,4.0,964981247,1
2,1,6,4.0,964982224,1
3,1,47,5.0,964983815,1
4,1,50,5.0,964982931,1


In [30]:
rating_df[rating_df['rating'] < 3.0]

,userId,movieId,rating,timestamp,encode
76,1,1219,2.0,964983393,-1
143,1,2253,2.0,964981775,-1
148,1,2338,2.0,964983546,-1
152,1,2389,2.0,964983094,-1
170,1,2617,2.0,964982588,-1
...,...,...,...,...,...
100766,610,135536,2.5,1493847638,-1
100772,610,136598,2.5,1493847366,-1
100778,610,138610,1.5,1493848976,-1
100797,610,145951,0.5,1493849019,-1


In [32]:
movies_df = pd.read_csv('ml-latest-small/movies.csv')
col_names = movies_df['movieId'].values.reshape(-1,)
num_row = rating_df['userId'].values[-1]
dataset_df = pd.DataFrame(index = range(0, num_row), columns = col_names) # create dataset, each column corresponding to a movieId
dataset_df = dataset_df.fillna(-1)

for i in range(num_row):
    tmp_df = rating_df[rating_df['userId'] == i+1]
    for _, row in tmp_df.iterrows():
        dataset_df.at[i, row['movieId']] = row['encode'] 

In [33]:
dataset_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
606,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
607,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
608,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [34]:
dataset_df.iloc[0].head(100)

1      1
2     -1
3      1
4     -1
5     -1
      ..
107   -1
108   -1
110    1
111   -1
112   -1
Name: 0, Length: 100, dtype: int64

### 2. Set aside 10% of your dataset for testing. How many users are there in the training set? What is the storage capacity of the network? Is the network likely to be able to store the dataset?

In [ ]:
train_set, test_set = train_test_split(dataset_df.values, test_size=0.1, random_state=42, shuffle=False)

In [ ]:
train_set.shape

(549, 9742)

In [ ]:
test_set.shape

(61, 9742)

In [ ]:
train_set[548]

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

#### How many users are there in the training set?
- There are 549 users

#### What is the storage capacity of the network?

In [ ]:
d = len(dataset_df.columns) # number of features/neurons
print(f'd: {d}')
# the size of training data that a Hopfield network with d visible units can store without causing errors
# in associative recall. This is known as storage capacity of a Hopfield network (p. 240)
print(f'The storage capacity: {round(0.15 * d)} training examples')

d: 9742
The storage capacity: 1461 training examples


#### Is the network likely to be able to store the dataset?

In [ ]:
print('The total number of bits that Hopfield network can store: ')
print(f'{round(0.15 * d * d)} bits')

The total number of bits that Hopfield network can store: 
14235985 bits


In [ ]:
print('The number of weights in the network is:')
print((d*(d-1))/2)

The number of weights in the network is:
47448411.0


In [ ]:
print('The dataset contains: ')
print(f'{610 * 9742} bits')

The dataset contains: 
5942620 bits


Yes, the network is likely to able to store the dataset because the storage capacity can contain 1461 examples in which can be memorized from a Hopfield network. Furthermore, there are only 610 examples in our dataset.

### 3. Construct a Hopfield network for your dataset and train it on your training set.
- Hint: You may want to take a look at the pseudocode on pp. 66-67 of Artificial Intelligence Engines.
- What accuracy does your trained network achieve?


In [ ]:
def learning(data):
    row, col = data.shape
    LEARNING_RATE = 1/row
    
    # initialize the weights W to zero
    W = np.zeros(shape=(col, col))
    b = np.zeros(col)
    b = np.full(col, -1) # AI Engines textbook suggestion
    
    # calculate the weight matrix, following line is the same as the for-loop below
#     W = np.matmul(data.T, data)
    
    for i in tqdm(range(row), desc='learning'):
        # find weights for the vector x_t
#         W_t = (np.outer(data[i], data[i]))
        W_t = LEARNING_RATE * (np.outer(data[i], data[i]))

        # update weights and bias
        W += W_t
#         b += data[i] # main textbook suggestions
    np.fill_diagonal(W, 0)
    return [W, b]

In [ ]:
W, b = learning(train_set)
W

learning: 100%|██████████| 549/549 [05:51<00:00,  1.56it/s]


array([[0.        , 0.41347905, 0.40619308, ..., 0.36247723, 0.36247723,
        0.36247723],
       [0.41347905, 0.        , 0.67941712, ..., 0.71584699, 0.71584699,
        0.71584699],
       [0.40619308, 0.67941712, 0.        , ..., 0.85428051, 0.85428051,
        0.85428051],
       ...,
       [0.36247723, 0.71584699, 0.85428051, ..., 0.        , 1.        ,
        0.99271403],
       [0.36247723, 0.71584699, 0.85428051, ..., 1.        , 0.        ,
        0.99271403],
       [0.36247723, 0.71584699, 0.85428051, ..., 0.99271403, 0.99271403,
        0.        ]])

In [ ]:
W.shape

(9742, 9742)

In [ ]:
W[W < 0]

array([-0.0273224 , -0.00910747, -0.03096539, ..., -0.0273224 ,
       -0.0273224 , -0.02003643])

In [ ]:
b

array([-1, -1, -1, ..., -1, -1, -1])

In [ ]:
def recall(W, b, corrupted_x):
    state_y = np.copy(corrupted_x)
    stable = False
    K = len(state_y)
    while not stable:
        stable = True
        for _ in range(K+1):
            i = np.random.choice(K, replace=False)
            u_j = np.dot(W[i], state_y) + b[i]
#             print(u_j)
            state_y_last = np.copy(state_y[i])
            state_y[i] = np.sign(u_j)
            if state_y[i] != state_y_last:
                stable = False

    return state_y

In [ ]:
pred = recall(W, b, train_set[0])
print(f'train_set[0]: {train_set[0]}')
print(f'pred:\n{pred}')
print('\n')
print(f'indices of the unit state having value of 1 in input:\n{np.where(train_set[0] >= 0)}')
print('\n')
print(f'indices of the unit state having value of 1 in prediction:\n{np.where(pred >= 0)}')
print('\n')
print(f'is the output similar to the input?\n{np.array_equal(train_set[0], pred)}')

train_set[0]: [ 1 -1  1 ... -1 -1 -1]
pred:
[-1 -1 -1 ... -1 -1 -1]


indices of the unit state having value of 1 in input:
(array([   0,    2,    5,   43,   46,   62,   89,   97,  124,  130,  136,
        184,  190,  197,  201,  224,  257,  275,  291,  307,  314,  320,
        325,  367,  384,  398,  418,  436,  461,  476,  484,  485,  508,
        509,  510,  513,  520,  546,  551,  559,  592,  594,  615,  632,
        701,  705,  720,  723,  734,  767,  781,  782,  783,  786,  787,
        788,  789,  797,  801,  810,  815,  820,  828,  829,  831,  836,
        856,  863,  898,  899,  900,  907,  909,  911,  914,  915,  921,
        923,  925,  927,  939,  955,  957,  964,  969,  974,  977,  981,
        990,  996, 1036, 1060, 1076, 1084, 1110, 1126, 1146, 1154, 1171,
       1181, 1183, 1190, 1218, 1220, 1224, 1235, 1261, 1298, 1319, 1326,
       1332, 1333, 1401, 1407, 1431, 1444, 1475, 1480, 1487, 1493, 1503,
       1505, 1516, 1517, 1522, 1526, 1543, 1553, 1556, 1557, 1559, 1562,

In [ ]:
corrupted_x = np.copy(train_set[0])
print(f'train_set[0]: {train_set[0]}')
corrupted_x[-1] = 1
print(f'corrupted_x: {corrupted_x}')
pred = recall(W, b, corrupted_x)
print(f'pred: {pred}')
print(f'is the output similar to the input?\n{np.array_equal(train_set[0], pred)}')
print(f'indices of the unit state having value of 1 in prediction:\n{np.where(pred >= 0)}')

train_set[0]: [ 1 -1  1 ... -1 -1 -1]
corrupted_x: [ 1 -1  1 ... -1 -1  1]
pred: [-1 -1 -1 ... -1 -1 -1]
is the output similar to the input?
False
indices of the unit state having value of 1 in prediction:
(array([277], dtype=int64),)


In [ ]:
def predict(dataset):
    correct = 0

    for x in tqdm(dataset, desc='Predicting'):
        pred = recall(W, b, x)
        correct += np.count_nonzero(pred==x)
    row, col = dataset.shape
    return (correct/(row*col) * 100)

In [ ]:
accuracy = predict(train_set)
print(f'\nAccuracy: {accuracy}')

Predicting: 100%|██████████| 549/549 [2:08:49<00:00, 14.08s/it]  


Accuracy: 98.7056214262396


### 4. Determine your network’s accuracy on the test set. How well is the network performing?


In [ ]:
accuracy = predict(test_set)
print(f'\nAccuracy: {accuracy}')

Predicting: 100%|██████████| 61/61 [14:18<00:00, 14.07s/it]


Accuracy: 97.93003759284625


In [ ]:
train_set.size

5348358

In [ ]:
row_train, col_train = train_set.shape
row_train * col_train

5348358

### 5. Choose a few higher-performing examples from the test set, then use movies.csv to determine which movies those users liked.
### Which other movies did the network predict that those users might like? Do the recommendations seem reasonable?


In [ ]:
# find higher-performing examples
user_has_the_most_movies = 0
user_idx = 0
for idx, user in enumerate(test_set):
    num_of_watched_movies = len(np.where(user >= 0)[0])
    if user_has_the_most_movies < num_of_watched_movies:
        user_has_the_most_movies = num_of_watched_movies
        user_idx = idx
                                
print(user_has_the_most_movies)
print(user_idx)

1159
60


In [ ]:
user_movie_dict = defaultdict(list)
for idx, user in enumerate(test_set):
    num_of_watched_movies = len(np.where(user >= 0)[0])
    user_movie_dict[num_of_watched_movies].append(idx)
    
pprint.pprint(user_movie_dict)
    
res = heapq.nlargest(3, user_movie_dict)
print(f'the most movies: {res}')
print('user has the most watched or high rating movies:')
for x in res:
    print(f'user_idx: {user_movie_dict[x]}')

defaultdict(<class 'list'>,
            {12: [26],
             15: [48],
             19: [45],
             20: [19],
             21: [24],
             22: [18],
             23: [28],
             25: [7],
             27: [0, 15],
             28: [25],
             31: [6],
             37: [59],
             38: [39],
             39: [41],
             40: [31],
             45: [38],
             46: [33],
             51: [8, 21],
             52: [32],
             60: [35],
             63: [16],
             68: [43],
             69: [29],
             75: [4],
             77: [34],
             83: [3],
             87: [42],
             96: [54],
             101: [51],
             110: [1],
             118: [52],
             123: [9],
             136: [17],
             137: [2],
             142: [14, 22],
             148: [27],
             156: [37],
             163: [20],
             174: [57],
             176: [13],
             180: [55],
             

In [ ]:
pred = recall(W, b, test_set[60])
print(np.where(pred >= 0))

(array([277], dtype=int64),)


In [ ]:
user_60 = movies_df.iloc[np.where(test_set[60] >= 0)[0]]
user_60.to_csv('user_60_rating_movies.csv')
user_60

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,6,Heat (1995),Action|Crime|Thriller
15,16,Casino (1995),Crime|Drama
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
43,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
...,...,...,...
9434,166534,Split (2017),Drama|Horror|Thriller
9461,168248,John Wick: Chapter Two (2017),Action|Crime|Thriller
9462,168250,Get Out (2017),Horror
9463,168252,Logan (2017),Action|Sci-Fi


In [ ]:
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


#### As shown above, the prediction seems to be reasonable for the user (60) because he or she seems to like the movies with genres such as crime and drama. But, there is only a recommendation. This is very strange.

In [ ]:
pred = recall(W, b, test_set[49])
print(np.where(pred >= 0))

(array([ 277,  418, 1291], dtype=int64),)


In [ ]:
user_49 = movies_df.iloc[np.where(test_set[49] >= 0)[0]]
user_49.to_csv('user_49_rating_movies.csv')
user_49

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,6,Heat (1995),Action|Crime|Thriller
9,10,GoldenEye (1995),Action|Adventure|Thriller
15,16,Casino (1995),Crime|Drama
16,17,Sense and Sensibility (1995),Drama|Romance
...,...,...,...
9644,179817,Darkest Hour (2017),Drama|War
9647,180031,The Shape of Water (2017),Adventure|Drama|Fantasy
9653,180297,The Disaster Artist (2017),Comedy|Drama
9660,181315,Phantom Thread (2017),Drama|Romance


In [ ]:
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
418,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
1291,1721,Titanic (1997),Drama|Romance


#### Similar to user (60), the prediction for user (49) only shows three recommendations. The recommendations seem somewhat reasonable to the user 

In [ ]:
pred = recall(W, b, test_set[56])
print(np.where(pred >= 0))

(array([277], dtype=int64),)


In [ ]:
user_56 = movies_df.iloc[np.where(test_set[56] >= 0)[0]]
user_56.to_csv('user_56_rating_movies.csv')
user_56

,movieId,title,genres
14,15,Cutthroat Island (1995),Action|Adventure|Romance
16,17,Sense and Sensibility (1995),Drama|Romance
17,18,Four Rooms (1995),Comedy
27,28,Persuasion (1995),Drama|Romance
28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
...,...,...,...
7212,72998,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
7302,76093,How to Train Your Dragon (2010),Adventure|Animation|Children|Fantasy|IMAX
7767,91500,The Hunger Games (2012),Action|Adventure|Drama|Sci-Fi|Thriller
8002,97225,Hotel Transylvania (2012),Animation|Children|Comedy


In [ ]:
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


#### Interestingly, we observe that all the users have the same recommendation which is the movie "Shawshank Redemption"

### 6. Construct a feature vector for yourself, choosing several movies that you’ve seen and liked. (Do not choose all movies that you have seen -- leave room for recommendations.) What other movies does the network recommend for you? How many of the recommended movies have you actually seen? Were the recommendations accurate?

In [ ]:
# initialize array with value -1
my_test_sample = np.full((len(test_set[0]),), -1)
my_test_sample

array([-1, -1, -1, ..., -1, -1, -1])

In [ ]:
my_movies = [1, 2, 362, 480]
for x in my_movies:
    my_test_sample[x-1] = 1

my_test_sample

array([ 1,  1, -1, ..., -1, -1, -1])

In [ ]:
my_pred = recall(W, b, my_test_sample)
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


### What other movies does the network recommend for you?
Shawshank Redemption, The (1994)

### How many of the recommended movies have you actually seen?
None

### Were the recommendations accurate?
No, the recommendations were not accurate